# HSE 2021: Mathematical Methods for Data Analysis

## Homework 4

### Attention!
* For tasks where <ins>text answer</ins> is required **Russian language** is **allowed**.
* If a task asks you to describe something (make coclusions) then **text answer** is **mandatory** and **is** part of the task
* **Do not** upload the dataset to the grading system (we already have it)
* We **only** accept **ipynb** notebooks. If you use Google Colab then you'll have to download the notebook before passing the homework
* **Do not** use python loops instead of NumPy vector operations over NumPy vectors - it significantly decreases performance (see why https://blog.paperspace.com/numpy-optimization-vectorization-and-broadcasting/), will be punished with -0.25 for **every** task. 


### Contents

#### Decision Trees - 7 points
* [Task 1](#task1) (0.5 points)
* [Task 2](#task2) (0.5 points)
* [Task 3](#task3) (2 points)
* [Task 4](#task4) (0.5 points)
* [Task 5](#task5) (0.5 points)
* [Task 6](#task6) (2 points)
* [Task 7](#task7) (0.5 points)
* [Task 8](#task8) (0.5 points)

#### Ensembles - 3 points
* [Task 1](#task2_1) (1 point)
* [Task 2](#task2_2) (0.7 points)
* [Task 3](#task2_3) (0.5 points)
* [Task 4](#task2_4) (0.7 points)
* [Task 5](#task2_5) (0.1 points)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (11, 5)
%matplotlib inline

# Part 1. Decision Tree Regressor

In this task you will be implementing decision tree for the regression by hands. 

### Task 1 <a id="task1"></a> (0.5 points)

Implement the function `H()` which calculates impurity criterion. We will be training regression tree, therefore, impurity criterion will be variance.

* You cannot use loops
* If `y` is empty, the function should return 0

In [ ]:
def H(y):
    """
    Calculate impurity criterion
    
    Parameters
    ----------
    y : np.array
        array of objects target values in the node

    Returns
    -------
    H(R) : float
        Impurity in the node (measuread by variance)
    """
    # YOUR CODE HERE
    return 0 if len(y) == 0 else np.sum((y - np.mean(y)) ** 2) / len(y)


In [ ]:
# Test the function
assert np.allclose(H(np.array([4,2,2, 2])), 0.75)
assert np.allclose(H(np.array([])), 0.0)

### Task 2 <a id="task2"></a>  (0.5 points)

To find the best split in the node we need to calculate the cost function. Denote: 
- `R` all the object in the node
- `j` index of the feature selected for the split
- `t` threshold
- `R_l` and `R_r` objects in the left and right child nodes correspondingly

We get the following cost function:

$$
Q(R, j, t) =\frac{|R_\ell|}{|R|}H(R_\ell) + \frac{|R_r|}{|R|}H(R_r) \to \min_{j, t},
$$

Implement the function `Q`, which should calculate value of the cost function for a given feature and threshold.

In [ ]:
def Q(X, y, j, t):
    """
    Calculate cost function
    Parameters
    ----------
    X : ndarray
        array of objects in the node 
    y : ndarray
        array of target values in the node 
    j : int
        feature index (column in X)
    t : float
        threshold

    Returns
    -------
    Q : float
        Value of the cost function
    """   
    # YOUR CODE HERE
    R_l = y[X[:, j] <= t]
    R_r = y[X[:, j] > t]
    return len(R_l) * H(R_l) / len(y) + len(R_r) * H(R_r) / len(y)

### Task 3 <a id="task3"></a>  (2 points)

Now, let's implement `MyDecisionTreeRegressor` class. More specifically, you need to implement the following methods:

- `best_split`
- `grow_tree`
- `get_prediction`

Read docstrings for more details. Do not forget to use function `Q` implemented above, when finding the `best_split`

In [ ]:
class Node(object):
    """
    Class for a decision tree node.
    
    Parameters
    ----------
    right : Node() or None
        Right child
    right : Node() or None
        Left child
    threshold: float
        
    column: int
        
    depth: int
        
    prediction: float
        prediction of the target value in the node (average values calculated on a train dataset)
    is_terminal:bool
        indicates whether it is a terminal node (leaf) or not
    """    
    def __init__(self):        
        self.right = None
        self.left = None
        self.threshold = None
        self.column = None
        self.depth = None
        self.is_terminal = False
        self.prediction = None
        
    def __repr__(self):
        if self.is_terminal:
            node_desc = 'Pred: {:.2f}'.format(self.prediction)
        else:
            node_desc = 'Col {}, t {:.2f}, Pred: {:.2f}'.format(self.column, self.threshold, self.prediction)
        return node_desc

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class MyDecisionTreeRegressor(RegressorMixin, BaseEstimator):
    """
    Class for a Decision Tree Regressor.

    Parameters
    ----------
    max_depth : int
        Max depth of a decision tree.
    min_samples_split : int
        Minimal number of samples (objects) in a node to make a split.
    """ 
    def __init__(self, max_depth=3, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
    
    def _more_tags(self):
        return {
            'requires_y': False
        }
            
    def best_split(self, X, y):
        """
        Find the best split in terms of Q of data in a given decision tree node. 
        Try all features and thresholds. 
        
        Parameters
        ----------
        X : ndarray, shape (n_objects, n_features)
            Objects in the parent node
        y : ndarray, shape (n_objects, )
            1D array with the object labels. 
            
        Returns
        -------
        best_split_column : int
            Index of the best split column
        best_threshold : float
            The best split condition.
        X_left : ndarray, shape (n_objects_l, n_features)
            Objects in the left child
        y_left : ndarray, shape (n_objects_l, )
            Objects labels in the left child. 
        X_right : ndarray, shape (n_objects_r, n_features)
            Objects in the right child
        y_right : ndarray, shape (n_objects_r, )
            Objects labels in the right child. 
        """
        
        # To store best split parameters
        best_split_column = None
        best_threshold = None
        # without splitting
        best_cost = H(y) 
        best_information_gain = -999
        
        # YOUR CODE HERE
        for split_column in range(X.shape[1]):
            
            # Select values of the column
            x_col = X[:, split_column]
            
            # For each value in the column ...
            for i_x in range(0, len(x_col)):
                
                # Take the value as a threshold for a split
                threshold = x_col[i_x]
                
                # Calculate information gain of the split
                information_gain = best_cost - Q(X, y, split_column, threshold)
                # information_gain = 0.2 (example)
                
                # Is this information_gain the best?
                if information_gain > best_information_gain:
                    best_split_column = split_column
                    best_threshold = threshold
                    best_information_gain = information_gain
                    
        # If no split available
        if best_information_gain == -999:
            return None, None, None, None, None, None
        
        # Take the best split parameters and make this split
        x_col = X[:, best_split_column]
        X_left = X[x_col <= best_threshold, :]
        y_left = y[x_col <= best_threshold]
        X_right = X[x_col > best_threshold, :]
        y_right = y[x_col > best_threshold]
        
        return best_split_column, best_threshold, X_left, y_left, X_right, y_right
    
    def is_terminal(self, node, y):
        """
        Check terminality conditions based on `max_depth` and `min_samples_split` parameters for a given node. 
        
        Parameters
        ----------
        node : Node, 
            
        y : ndarray, shape (n_objects, )
            Object labels. 
            
        Returns
        -------
        Is_termial : bool
            If True, node is terminal
        """
        if node.depth >= self.max_depth:    
            return True
        if len(y) < self.min_samples_split:   
            return True
        return False
        
    def grow_tree(self, node, X, y):
        """
        Reccurently grow the tree from the `node` using a `X` and `y` as a dataset:
         - check terminality conditions
         - find best split if node is not terminal
         - add child nodes to the node
         - call the function recursively for the added child nodes
        
        Parameters
        ----------
        node : Node() object
            Current node of the decision tree.
        X : ndarray, shape (n_objects, n_features)
            Objects 
        y : ndarray, shape (n_objects)
            Labels
        """
        
        if self.is_terminal(node, y):
            node.is_terminal =True
            return
                
        # YOUR CODE HERE
        
        if len(np.unique(y)) == 1:
            node.is_terminal = True
            return
        
        # Make best split
        split_column, threshold, X_left, y_left, X_right, y_right = self.best_split(X, y) # Make a split
        
        # Check additional termination conditions
        if split_column is None:
            node.is_terminal = True
            return
        
        
        # Add split parameters into the current node
        node.column = split_column
        node.threshold = threshold
        
        # Create a left child of the current node
        node.left = Node()
        node.left.depth = node.depth + 1
        node.left.prediction = np.mean(y_left)
        
        # Create a right child of the current node
        node.right = Node()
        node.right.depth = node.depth + 1
        node.right.prediction = np.mean(y_right)
        
        # Make splits for the left and right nodes
        self.grow_tree(node.right, X_right, y_right)
        self.grow_tree(node.left, X_left, y_left)
        

    def fit(self, X, y):
        """
        Fit the Decision Tree Regressor.
            
        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            The input samples.
        y : ndarray, shape (n_samples,) or (n_samples, n_outputs)
            The target values.
        Returns
        -------
        self : object
            Returns self.
        """
        # Check the input
        if y is None:
            raise ValueError('Y should not be None')
        X, y = check_X_y(X, y, accept_sparse=False)
        self.is_fitted_ = True
        self.n_features_in_ = X.shape[1]
        
        # Initialize the tree (root node, depth = 0)
        self.tree_ = Node()                             
        self.tree_.depth = 0                            
        self.tree_.prediction = np.mean(y)
        
        # Grow the tree
        self.grow_tree(self.tree_, X, y)
        return self        
    
    def get_prediction(self, node, x):
        """
        Get prediction for an object `x`
            - Return prediction of the `node` if it is terminal
            - Otherwise, recursively call the function to get predictions of the proper child
        
        Parameters
        ----------
        node : Node() object
            Current node of the decision tree.
        x : ndarray, shape (n_features,)
            Array of feature values of one object.
        Returns
        -------
        y_pred : float
            Prediction for an object x
        """
        # YOUR CODE HERE
        
        if node.is_terminal:
            return node.prediction
        return self.get_prediction(node.right, x) if x[node.column] > node.threshold else self.get_prediction(node.left, x)
    
    def predict(self, X):
        """ 
        Get prediction for each object in X
        
        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            Returns predictions.
        """
        # Check input and that `fit` had been called
        X = check_array(X, accept_sparse=False)
        check_is_fitted(self, 'is_fitted_')
        
        # Get predictions
        y_predicted = []
        for x in X:
            y_curr = self.get_prediction(self.tree_, x)
            y_predicted.append(y_curr)
        return np.array(y_predicted)

In [ ]:
# check yourself
from sklearn.utils.estimator_checks import check_estimator

check_estimator(MyDecisionTreeRegressor())

### Task 4 <a id="task4"></a>  (0.5 points)

Load boston dataset and split it on the train ($70\%$) and test ($30\%$). Fit Decision Tree of depth 1 and make the following plot:

- Scatter plot of the traning points (selected for split feature on the x-axis, target variable on the y-axis)
- Fitted model 

P.S. Depth of the tree is equal to the longest path from the root note to the leaf. Thus, tree with depth 1 will have exactly 1 split. 

P.P.S. Both fitted model and the training points should be on the same plot

In [19]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

# YOUR CODE HERE

data = load_boston()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size = 0.3, random_state = 42)

In [ ]:
clf = MyDecisionTreeRegressor(max_depth=1)
clf.fit(X_train, y_train)

In [ ]:
print(clf.tree_)
print(clf.tree_.column)
print(clf.tree_.threshold)
print(clf.tree_.right)
print(clf.tree_.left)

In [ ]:
X_l = X_train[X_train[:, clf.tree_.column] <= clf.tree_.threshold]
y_l = y_train[X_train[:, clf.tree_.column] <= clf.tree_.threshold]
X_r = X_train[X_train[:, clf.tree_.column] > clf.tree_.threshold]
y_r = y_train[X_train[:, clf.tree_.column] > clf.tree_.threshold]

y_pred = clf.predict(X_train)
y_l_pred = y_pred[X_train[:, clf.tree_.column] <= clf.tree_.threshold]
y_r_pred = y_pred[X_train[:, clf.tree_.column] > clf.tree_.threshold]

In [ ]:
plt.scatter(X_l[:, clf.tree_.column], y_l, color='blue')
plt.scatter(X_r[:, clf.tree_.column], y_r, color='green')
plt.scatter(X_l[:, clf.tree_.column], y_l_pred, color = 'orange')
plt.scatter(X_r[:, clf.tree_.column], y_r_pred, color = 'red')
plt.axvline(clf.tree_.threshold, color = 'maroon')
plt.title('Train')

In [ ]:
X_l = X_test[X_test[:, clf.tree_.column] <= clf.tree_.threshold]
y_l = y_test[X_test[:, clf.tree_.column] <= clf.tree_.threshold]
X_r = X_test[X_test[:, clf.tree_.column] > clf.tree_.threshold]
y_r = y_test[X_test[:, clf.tree_.column] > clf.tree_.threshold]

y_pred = clf.predict(X_test)
y_l_pred = y_pred[X_test[:, clf.tree_.column] <= clf.tree_.threshold]
y_r_pred = y_pred[X_test[:, clf.tree_.column] > clf.tree_.threshold]

In [ ]:
plt.scatter(X_l[:, clf.tree_.column], y_l, color='blue')
plt.scatter(X_r[:, clf.tree_.column], y_r, color='green')
plt.scatter(X_l[:, clf.tree_.column], y_l_pred, color = 'orange')
plt.scatter(X_r[:, clf.tree_.column], y_r_pred, color = 'red')
plt.axvline(clf.tree_.threshold, color = 'maroon')
plt.title('Test')

### Task 5 <a id="task5"></a>  (0.5 points)

Keep working with boston dataset. 
- Use `GridSearchCV` to find the best hyperparameters (`max_depth` and `min_samples_split`) on 5-Fold cross-validation
- Train the model with the best set of hyperparameters on the whole train dataset. 
- Report `RMSE` on test dataset and hyperparameters of the best estimator. 

In [ ]:
# YOUR CODE HERE
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': range(1, 10), 'min_samples_split': range(1, 15)}

clf = MyDecisionTreeRegressor()
gs = GridSearchCV(clf, parameters, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

print(f"Best parameters: {gs.best_params_}")
print(f"Best RMSE: {-1 * gs.best_score_}")

### Task 6 <a id="task6"></a>  (2 points)

Recall definition of bias and variance:
$$
\text{Bias}^2 = \mathbb{E}_{p(x, y)} \left[  (f(x) - \mathbb{E}_{\mathbb{X}}a_{\mathbb{X}}(x))^2 \right] \\
\text{Variance} = \mathbb{E}_{p(x, y)} \left[  \mathbb{V}_{\mathbb{X}}( a_{\mathbb{X}}(x))  \right]
$$

We wil now use use the following algorithm to estimate bias^2 and variance:

1. Use bootsrap to create `n_iter` samples from the original dataset: $X_1, \dots, X_{n\_iter}$. Each $X_i$ has $N$ observations (randomly selected from the original dataset with replacement)
2. For each bootstrapped sample define out-of-bag (OOB) sample $Z_1, \dots, Z_{n\_iter}$, which contains all the observations, which did not appear in the corresponding boostraped sample
3. Fit the model on observations from $X_i$s and compute predictions on points from $Z_i$s
4. For a given *object* $x_n$:
     - bias^2: squared difference between true value $y_n$ and average prediction (average over the algorithms, for which $x_n$ was in OOB)
     - variance: variance of the predictions (predictions of the algorithms, for which $x_n$ was in OOB)
5. Average bias^2 and variance over all the points


**Consider a toy example.** You are given a dataset with 5 observations: $((x_1 ,y_1), (x_2, y_2), (x_3, y_3), (x_4, y_4), (x_5, y_5))$, where $x_i$ is a vector of features, $y_i$ is a target variable . And choose `n_iter` to be 3. 
* Example of bootstrapped samples:
$$X_1 = (x_2, x_5, x_2, x_3, x_2, x_5)$$
$$X_2 = (x_5, x_2, x_4, x_4, x_1, x_5)$$
$$X_3 = (x_1, x_3, x_1, x_4, x_3, x_1)$$

* Corresponding OOB samples:
$$Z_1 = (x_1, x_4)$$
$$Z_2 = (x_3)$$
$$Z_3 = (x_2, x_5)$$

* Fit models using $X_1$, $X_2$ and $X_3$ as training data. Use 1st model to make predictions for points from $Z_1$, second - for $Z_2$, etc. and use these predictions to estimate bias and variance. 

    
**Implement `get_bias_variance` function, using the algorithm above**

*Note:*  You can only use 1 loop (for bootsrap iterations `n_iter`). All other operations should be vectorized. 

P.S. These numpy functions might be usefull here `np.nanmean`, `np.nanstd` (but you are not obliged to use them). 

In [ ]:
def get_bias_variance(estimator, x, y, n_iter):
    """ 
    Calculate bias^2 and variance of the `estimator`. Using a given dataset and bootstrap with `n_iter` samples. 

    Parameters
    ----------
    x : ndarray, shape (n_samples, n_features)
        The input samples.
    y : ndarray, shape (n_samples, n_features)
        The input samples.
    n_iter: int
        Number of samples in 
    Returns
    -------
    bias2 : float, 
        Estiamted squared bias
    variance : float, 
        Estiamted variance
    """
    
    # YOUR CODE HERE
    np.random.seed(42)
    all_preds = np.repeat(np.nan, y.shape[0]).reshape(y.shape[0], 1)
    for i in range(0, n_iter):
        train_indices = np.random.choice(y.shape[0], y.shape[0])
        while np.unique(train_indices).shape[0] == y.shape[0]:
            train_indices = np.random.choice(y.shape[0], y.shape[0])
        test_indices = np.arange(y.shape[0])
        test_indices = test_indices[np.isin(test_indices, train_indices, invert=True)]
        
        x_i = x[train_indices, :]
        y_x_i = y[train_indices]
        z_i = x[test_indices, :]
        y_z_i = y[test_indices]
        
        estimator.fit(x_i, y_x_i)
        y_z_i_pred = estimator.predict(z_i)
        
        pred_with_nans = np.repeat(np.nan, y.shape[0])
        np.put(pred_with_nans, test_indices, y_z_i_pred)
        pred_with_nans = pred_with_nans.reshape(y.shape[0], 1)
        all_preds = np.hstack((all_preds, pred_with_nans))
    all_preds = all_preds[:, 1:]
    return np.nanmean(np.square(y - np.nanmean(all_preds, axis=1))), np.nanmean(np.nanstd(all_preds, axis=1) ** 2)

In [ ]:
# Test
estimator = MyDecisionTreeRegressor(max_depth=7, min_samples_split=14)

get_bias_variance(estimator, X_train, y_train, 10)

### Task 7 <a id="task7"></a>  (0.5 points = 0.25 for code + 0.25 for comments)

Compute bias and variance for the trees of different depths. Plot how bias and variance change as depth increases. 

Comment on what you observe, how does your result correspond to what we have discussed in class?

In [ ]:
# YOUR CODE HERE
bias2_lst = []
var_lst = []
for max_depth in range(1, 10):
    clf = MyDecisionTreeRegressor(max_depth=max_depth, min_samples_split=14)
    bias2, var = get_bias_variance(clf, X_train, y_train, 10)
    bias2_lst.append(bias)
    var_lst.append(var)

In [ ]:
plt.plot(range(1, 10), bias_lst, color='blue', label='bias')
plt.plot(range(1, 10), var_lst, color='green', label='variance')
plt.xlabel('max depth')
plt.ylabel('values')
plt.title('Values of bias and variance depending on max depth')
plt.legend()

```Как мы видим, с ростом глубины значительно уменьшается bias, что свидетельствует об улучшении точности модели. Увеличение variance говорит об увеличении зависимости от исходных, причём по сравнению со значением при глубине 3 значение при глубине 7 выглядит достаточно большим.```

### Task 8 <a id="task8"></a>  (0.5 points = 0.25 for code + 0.25 for comments)

Let's try to reduce variance with bagging. Use `sklearn.ensemble.BaggingRegressor` to get an ensemble and compute its bias and variance. 

Answer the following questions:
 - How bagging should affect bias and variance in theory?
 - How bias and variance change (if they change) compared to an individual tree in you experiments? 
 - Do your results align with the theory? Why?

In [ ]:
from sklearn.ensemble import BaggingRegressor

# YOUR CODE HERE
model = BaggingRegressor(base_estimator=MyDecisionTreeRegressor(max_depth=7, min_samples_split=14), 
                          n_jobs=-1, n_estimators=100, random_state=42)
get_bias_variance(model, X_train, y_train, 10)

```Согласно теории, при использовании bagging усредняется некоторое количество деревьев, что должно уменьшить влияние конкретных данных на предсказание, то есть значительно уменьшить variance. Bias при этом, как правило, тоже уменьшается, но может и увеличиться (к примеру, если значение varianca изначально было одним из наилучших возможных). В нашем случае variance уменьшается крайне значительно, и неплохо уменьшается bias, что свидетельствует о согласованности с теорией.```

# Part 2. More Ensembles

In this part we will be working with [Thyroid Disease Data Set](https://archive.ics.uci.edu/ml/datasets/thyroid+disease) to solve a classification task. 

In [20]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('C:/Users/Александр/DataspellProjects/HSE-ML/fourth_hw/datasets/thyroid_disease.csv')

le = LabelEncoder()
y = le.fit_transform(df['Class'])
X = df.drop('Class', axis=1)
X.head(5)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,T3,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source
0,41.0,F,f,f,f,f,f,f,f,f,...,2.5,t,125.0,t,1.14,t,109.0,f,NaN,SVHC
1,23.0,F,f,f,f,f,f,f,f,f,...,2.0,t,102.0,f,NaN,f,NaN,f,NaN,other
2,46.0,M,f,f,f,f,f,f,f,f,...,NaN,t,109.0,t,0.91,t,120.0,f,NaN,other
3,70.0,F,t,f,f,f,f,f,f,f,...,1.9,t,175.0,f,NaN,f,NaN,f,NaN,other
4,70.0,F,f,f,f,f,f,f,f,f,...,1.2,t,61.0,t,0.87,t,70.0,f,NaN,SVI


### Task 1 <a id="task2_1"></a> (1 point)

Let's start with data preprocessing. 

0. Drop columns, which are not usefull (e.g. a lot of missing values). Motivate your choice. 
1. Split dataset into train and test
2. You've probably noticed that we have both categorical and numerical columns. Here is what you need to do with them:
    - Categorical: Fill missing values and apply one-hot-encoding (take a look at the argument `drop` in [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to figure out the best way to apply OHE to binary variables)
    - Numeric: Fill missing values
    
Use `ColumnTranformer` ([docs](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer)) to define a single transformer for all the columns in the dataset. It takes as input a list of tuples

```
ColumnTransformer([
    ('name1', transorm1, column_names1),
    ('name2', transorm2, column_names2)
])
```

Pay attention to an argument `remainder='passthrough'`. [Here](https://scikit-learn.org/stable/modules/compose.html#column-transformer) you can find some examples of how to use column transformer. 
    
Since we want to apply 2 transformations to categorical feature, it is very convenient to combine them into a `Pipeline`:

```
double_transform = make_pipeline(
                                 transform_1,
                                 transform_2
                                )
```

P.S. Choose your favourite way to fill missing values. 

*Hint* Categorical column usually have `dtype = 'object'`. This may help to obtain list of categorical and numerical columns in the dataset. 

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3771 non-null   float64
 1   sex                        3622 non-null   object 
 2   on_thyroxine               3772 non-null   object 
 3   query_on_thyroxine         3772 non-null   object 
 4   on_antithyroid_medication  3772 non-null   object 
 5   sick                       3772 non-null   object 
 6   pregnant                   3772 non-null   object 
 7   thyroid_surgery            3772 non-null   object 
 8   I131_treatment             3772 non-null   object 
 9   query_hypothyroid          3772 non-null   object 
 10  query_hyperthyroid         3772 non-null   object 
 11  lithium                    3772 non-null   object 
 12  goitre                     3772 non-null   object 
 13  tumor                      3772 non-null   objec

In [22]:
X = X.drop(columns=X.columns[X.notna().mean() < 3403.0 / 3772.0])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3771 non-null   float64
 1   sex                        3622 non-null   object 
 2   on_thyroxine               3772 non-null   object 
 3   query_on_thyroxine         3772 non-null   object 
 4   on_antithyroid_medication  3772 non-null   object 
 5   sick                       3772 non-null   object 
 6   pregnant                   3772 non-null   object 
 7   thyroid_surgery            3772 non-null   object 
 8   I131_treatment             3772 non-null   object 
 9   query_hypothyroid          3772 non-null   object 
 10  query_hyperthyroid         3772 non-null   object 
 11  lithium                    3772 non-null   object 
 12  goitre                     3772 non-null   object 
 13  tumor                      3772 non-null   objec

In [23]:
for column in X.columns:
    if X[column].dtype == 'object':
        print(column, pd.unique(X[column]))

sex ['F' 'M' nan]
on_thyroxine ['f' 't']
query_on_thyroxine ['f' 't']
on_antithyroid_medication ['f' 't']
sick ['f' 't']
pregnant ['f' 't']
thyroid_surgery ['f' 't']
I131_treatment ['f' 't']
query_hypothyroid ['f' 't']
query_hyperthyroid ['f' 't']
lithium ['f' 't']
goitre ['f' 't']
tumor ['f' 't']
hypopituitary ['f' 't']
psych ['f' 't']
TSH_measured ['t' 'f']
T3_measured ['t' 'f']
TT4_measured ['t' 'f']
T4U_measured ['t' 'f']
FTI_measured ['t' 'f']
TBG_measured ['f']
referral_source ['SVHC' 'other' 'SVI' 'STMW' 'SVHD']


In [24]:
X = X.drop(columns=['TBG_measured'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3771 non-null   float64
 1   sex                        3622 non-null   object 
 2   on_thyroxine               3772 non-null   object 
 3   query_on_thyroxine         3772 non-null   object 
 4   on_antithyroid_medication  3772 non-null   object 
 5   sick                       3772 non-null   object 
 6   pregnant                   3772 non-null   object 
 7   thyroid_surgery            3772 non-null   object 
 8   I131_treatment             3772 non-null   object 
 9   query_hypothyroid          3772 non-null   object 
 10  query_hyperthyroid         3772 non-null   object 
 11  lithium                    3772 non-null   object 
 12  goitre                     3772 non-null   object 
 13  tumor                      3772 non-null   objec

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer


# YOUR CODE HERE
# define column_transformer 
column_transformer = ColumnTransformer(transformers=[
    ('categorical', 
     make_pipeline(SimpleImputer(strategy='most_frequent'), 
                   OneHotEncoder(drop='if_binary')), 
     X.columns[X.dtypes == 'object']), 
    ('numeric', 
     SimpleImputer(strategy='mean'), 
     X.columns[X.dtypes != 'object'])], 
                                       remainder='passthrough')

# Transform the data
X_train = column_transformer.fit_transform(X_train)
X_test = column_transformer.transform(X_test)

### Task 2 <a id="task2_2"></a> (0.7 points  = 0.4 for code + 0.3 for comments)

Fit and compare 5 different models (use sklearn): Gradient Boosting, Random Forest, Decision Tree, SVM, Logitics Regression
    
* Choose one classification metric and justify your choice .
* Compare the models using score on cross validation. Mind the class balance when choosing the cross validation. (You can read more about different CV strategies [here](https://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold))
* Which model has the best performance? Which models overfit or underfit?

In [27]:
# YOUR CODE HERE
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import accuracy_score, f1_score

from statistics import mean

models = [GradientBoostingClassifier(random_state=42), RandomForestClassifier(random_state=42),
         DecisionTreeClassifier(random_state=42), SVC(random_state=42),
         LogisticRegression(random_state=42)]
model_names = ['GradientBoostingClassifier', 'RandomForestClassifier',
         'DecisionTreeClassifier', 'SVC',
         'LogisticRegression']

for i in range(0, len(models)):
    accuracy_scores = []
    f1_scores = []
    for train, test in StratifiedKFold().split(X_train, y_train):
        models[i].fit(X_train[train], y_train[train])
        accuracy_scores.append(accuracy_score(models[i].predict(X_train[test]), y_train[test]))
        f1_scores.append(f1_score(models[i].predict(X_train[test]), y_train[test]))
    print(f'{model_names[i]} test accuracy score estimate using Cross-Validation {mean(accuracy_scores):.4f}')
    print(f'{model_names[i]} actual accuracy test score: {accuracy_score(models[i].predict(X_test), y_test):.4f}')
    print(f'{model_names[i]} test f1 score estimate using Cross-Validation {mean(f1_scores):.4f}')
    print(f'{model_names[i]} actual f1 test score: {f1_score(models[i].predict(X_test), y_test):.4f}')
    print()

GradientBoostingClassifier test accuracy score estimate using Cross-Validation 0.9307
GradientBoostingClassifier actual accuracy test score: 0.9533
GradientBoostingClassifier test f1 score estimate using Cross-Validation 0.1566
GradientBoostingClassifier actual f1 test score: 0.3714

RandomForestClassifier test accuracy score estimate using Cross-Validation 0.9332
RandomForestClassifier actual accuracy test score: 0.9480
RandomForestClassifier test f1 score estimate using Cross-Validation 0.1730
RandomForestClassifier actual f1 test score: 0.3288

DecisionTreeClassifier test accuracy score estimate using Cross-Validation 0.9021
DecisionTreeClassifier actual accuracy test score: 0.9109
DecisionTreeClassifier test f1 score estimate using Cross-Validation 0.2712
DecisionTreeClassifier actual f1 test score: 0.3226

SVC test accuracy score estimate using Cross-Validation 0.9360
SVC actual accuracy test score: 0.9470
SVC test f1 score estimate using Cross-Validation 0.0000
SVC actual f1 test

C:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

In [ ]:
print(np.unique(y))
print(list(y).count(0))
print(list(y).count(1))

```Как мы видим, при достаточно высокой accuracy, мы получаем крайне низкие значения f1. Связано это с тем, что f1 является средневзвешенным precision и recall, которые отражают точность попаданий по классу и процент угадываний определённого класса соответственно. Обратим внимание на предсказываемые классы: они крайне несбалансированны. Скорее всего, обученные модели практически всегда предсказывают класс '0', что даёт высокую accuracy (если всегда предсказывать класс '0', то можно получить accuracy = 3541 / (3541+231) ), но низкую f1 (так как recall в таком случае получается низким).
Что касается лучшей модели, то на тестовой выборке это Gradient Boosting. Недообученными оказались все модели, если опираться на f1, но хуже всего проявила себя SVC.```

### Task 3 <a id="task2_3"></a> (0.5 points = 0.4 for code + 0.1 for comments)

More Gradient Boosting. Choose one of the three popular boosting implementations (xgboost, lightgbm, catboost). Select hyperparameters (number of trees, learning rate, depth) on cross-validation and compare with the methods from the previous task. 



In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

parameters = {'num_trees': np.linspace(200, 1400, 7), 
              'learning_rate': np.logspace(-3, 0, 4),
              'depth': range(1, 10), 'random_state': [42]}

clf = CatBoostClassifier()
gs = GridSearchCV(clf, parameters, scoring='f1', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

In [ ]:
print(f"Best parameters: {gs.best_params_}")
print(f"Best f1-score: {gs.best_score_}")

In [ ]:
clf = CatBoostClassifier(depth=2, learning_rate=1.0, num_trees=600, random_state=42)
clf.fit(X_train, y_train)
print(f'Train accuracy score: {accuracy_score(clf.predict(X_train), y_train):.4f}')
print(f'Test accuracy score: {accuracy_score(clf.predict(X_test), y_test):.4f}')
print(f'Train f1 score: {f1_score(clf.predict(X_train), y_train):.4f}')
print(f'Test f1 score: {f1_score(clf.predict(X_test), y_test):.4f}')

```Как мы видим, данный метод показал один из лучших результатов по accuracy и даже показал неожиданно высокий результат по f1 на обучающей выборке, однако показал крайне низкий в целом (что свидетельствует о явном переобучении) и средний относительно других моделей.```

### Task 4 <a id="task2_4"></a> (0.7 points = 0.4 for code + 0.3 for comments)

Now let's train more fancy ensembles:

* Bagging with decision trees as base estimators
* Bagging with gradient boosting as base estimators (use large amount of trees in gradient boosting, >100)
* [Voting classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier) 
* [Stacking Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier) with Logistic Regression as a final model
* [Stacking Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier) with Gradeint Boosting as a final model


If not stated in the task, feel free to tune / choose hyperparameters and base models.

Answer the questions:
* Which model has the best performance?
* Does bagging reduce overfitting of the gradient boosting with large amount of trees? 
* What is the difference between voting and staking? 

In [ ]:
from sklearn.ensemble import BaggingClassifier

parameters = {'base_estimator__max_depth': range(1, 10), 'n_estimators': range(1, 10)}

clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42), random_state=42)
gs = GridSearchCV(clf, parameters, scoring='f1', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

print(f"Best parameters: {gs.best_params_}")
print(f"Best f1-score: {gs.best_score_}")

In [ ]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=6, random_state=42), 
                        n_estimators=4, random_state=42)
clf.fit(X_train, y_train)
print(f'Train accuracy score: {accuracy_score(clf.predict(X_train), y_train):.4f}')
print(f'Test accuracy score: {accuracy_score(clf.predict(X_test), y_test):.4f}')
print(f'Train f1 score: {f1_score(clf.predict(X_train), y_train):.4f}')
print(f'Test f1 score: {f1_score(clf.predict(X_test), y_test):.4f}')

In [ ]:
parameters = {'base_estimator__max_depth': range(1, 10), 
              'base_estimator__n_estimators': np.linspace(200, 1400, 7, dtype=int), 
              'n_estimators': range(1, 10)}

clf = BaggingClassifier(base_estimator=GradientBoostingClassifier(random_state=42), random_state=42)
gs = GridSearchCV(clf, parameters, scoring='f1', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

print(f"Best parameters: {gs.best_params_}")
print(f"Best f1-score: {gs.best_score_}")

In [ ]:
clf = BaggingClassifier(base_estimator=GradientBoostingClassifier(max_depth=5, n_estimators=400, random_state=42), 
                        n_estimators=4, random_state=42)
clf.fit(X_train, y_train)
print(f'Train accuracy score: {accuracy_score(clf.predict(X_train), y_train):.4f}')
print(f'Test accuracy score: {accuracy_score(clf.predict(X_test), y_test):.4f}')
print(f'Train f1 score: {f1_score(clf.predict(X_train), y_train):.4f}')
print(f'Test f1 score: {f1_score(clf.predict(X_test), y_test):.4f}')

In [ ]:
from sklearn.ensemble import VotingClassifier

parameters = {'voting': ['soft', 'hard']}

clf = VotingClassifier(estimators=[('GradientBoosting', GradientBoostingClassifier(random_state=42)), 
                                   ('DecisionTree', DecisionTreeClassifier(random_state=42)), 
                                   ('RandomForest', RandomForestClassifier(random_state=42))])
gs = GridSearchCV(clf, parameters, scoring='f1', cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

print(f"Best parameters: {gs.best_params_}")
print(f"Best f1-score: {gs.best_score_}")

In [ ]:
clf = VotingClassifier(estimators=[('GradientBoosting', GradientBoostingClassifier(random_state=42)), 
                                   ('DecisionTree', DecisionTreeClassifier(random_state=42)), 
                                   ('RandomForest', RandomForestClassifier(random_state=42))], 
                       voting='soft')
clf.fit(X_train, y_train)
print(f'Train accuracy score: {accuracy_score(clf.predict(X_train), y_train):.4f}')
print(f'Test accuracy score: {accuracy_score(clf.predict(X_test), y_test):.4f}')
print(f'Train f1 score: {f1_score(clf.predict(X_train), y_train):.4f}')
print(f'Test f1 score: {f1_score(clf.predict(X_test), y_test):.4f}')

In [ ]:
from sklearn.ensemble import StackingClassifier

clf = StackingClassifier(estimators=[('DecisionTree', DecisionTreeClassifier(random_state=42)), 
                                     ('RandomForest', RandomForestClassifier(random_state=42))],
                        final_estimator=LogisticRegression(random_state=42),
                        n_jobs=-1)
clf.fit(X_train, y_train)
print(f'Train accuracy score: {accuracy_score(clf.predict(X_train), y_train):.4f}')
print(f'Test accuracy score: {accuracy_score(clf.predict(X_test), y_test):.4f}')
print(f'Train f1 score: {f1_score(clf.predict(X_train), y_train):.4f}')
print(f'Test f1 score: {f1_score(clf.predict(X_test), y_test):.4f}')

In [ ]:
clf = StackingClassifier(estimators=[('DecisionTree', DecisionTreeClassifier(random_state=42)), 
                                     ('RandomForest', RandomForestClassifier(random_state=42))],
                        final_estimator=GradientBoostingClassifier(random_state=42),
                        passthrough=True,
                        n_jobs=-1)
clf.fit(X_train, y_train)
print(f'Train accuracy score: {accuracy_score(clf.predict(X_train), y_train):.4f}')
print(f'Test accuracy score: {accuracy_score(clf.predict(X_test), y_test):.4f}')
print(f'Train f1 score: {f1_score(clf.predict(X_train), y_train):.4f}')
print(f'Test f1 score: {f1_score(clf.predict(X_test), y_test):.4f}')

```Начнём с отличия между voting и stacking. Основное заключается в том, что в voting объединяются несколько разных моделей и предсказанное значение усредняется. В случае stacking предсказания основных моделей не усредняются и не возвращаются, а подаются на вход финальной модели, которая уже возвращает предсказания.
Перейдём к оценке моделей. Во-первых, все они серьёзно страдают от переобучения (в наибольшей степени - voting, в наименьшей - bagging с основной моделью DecisionTrees). Во-вторых, наилучшей моделью на тестовой выборке оказалась voting, несмотря на наибольшее переобучение. Однако, если учитывать переобучение, то необходимо обратить внимание на модель stacking с финальной моделью GradientBoosting - эта модель значительно меньше переобучена, а на тестовой выборке выдаёт не настолько же меньший результат. В-третьих, сравнивая модели, где используется GradientBossting, можно сказать что переобучение в них ослабло по сравнению с CatBoosting.```

### Task 5 <a id="task2_5"></a> (0.1 points)

Report the test score for the best model, that you were able to get on this dataset. 

```По итогу, наилучший результат на тестовой выборке, а именно - 3714, оказался у простой модели GradientBoosting из задания 2. Однако судить только по этому показателю было бы не вполне корректно. Дело в том, что на обучающей выборке показатель этой модели ещё в разы меньше, что, скорее всего, свидетельствует о том, что тестовая выборка просто оказалась более подходящей для полученной модели.
Если судить более целостно, а именно:
* учитывать неплохие показатели на обучающей выборке
* учитывать неплохие показатели на тестовой выборке
* учитывать наличие переобучения
то наилучшей моделью я бы назвал последнюю - stacking с финальной GradientBoosting, поскольку она показала неплохой, но не слишком завышенный (не слишком переобучилась) результат на обучающей выборке и один из лучших результатов на тестовой выборке. Также, стоит обратить внимание на модель voting, у которой лучший показатель на тестовой выборке, но также очень явное переобучение.```